In [4]:
!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 6.1 MB/s eta 0:00:00


In [8]:
import pandas as pd

# loading the data the model was trained on
data = pd.read_csv("/content/drive/MyDrive/machine_learning/SONATA.csv")
data.Datetime = pd.to_datetime(data.Datetime)
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-11-03 09:15:00+05:30,370.549988,374.450012,369.500000,372.750000,0,0.0,0
1,2025-11-03 10:15:00+05:30,372.750000,373.600006,371.549988,372.600006,25782,0.0,0
2,2025-11-03 11:15:00+05:30,372.950012,373.799988,372.399994,373.049988,16760,0.0,0
3,2025-11-03 12:15:00+05:30,373.049988,373.899994,372.799988,372.799988,16982,0.0,0
4,2025-11-03 13:15:00+05:30,372.700012,373.500000,371.899994,373.000000,39286,0.0,0


In [10]:
from backtesting import *
from backtesting.lib import crossover
from tensorflow import keras
from joblib import load
import numpy as np

class Trading(Strategy):
  def init(self):
    self.model = keras.models.load_model("/content/drive/MyDrive/machine_learning/models/finstreet_new.keras")
    self.scaler = load("/content/drive/MyDrive/machine_learning/models/min_max_scaler.joblib")
    self.buffer = 0.01
    # Since the model operates on a 12H window
    # we load the last 12 rows of the data and update it with the new prediction every time
    self.window_size = 12
    self.data = data[["open", "High", "Low", "Close"]][-self.window_size:]
    self.scaled_data = self.scaler.transform(self.data)
  def next(self):
    self.prediction = self.model.predict(self.scaled_data)
    self.scaled_data = np.concatenate(self.scaled_data[:, 1:, :], self.prediction.reshape(1, self.prediction.shape[0], self.prediction.shape[1]))
    self.pred_prices = self.scaler.inverse_transform(self.prediction)[0]

    current_price = self.data[0, -1, -1] # Closing price of last bar
    pred_o = self.pred_prices[0]
    pred_h = self.pred_prices[1]
    pred_l = self.pred_prices[2]
    pred_c = self.pred_prices[3]

    # checking if model is hallucinating
    if (pred_h < pred_o or pred_h < pred_c or pred_l > pred_o or pred_l > pred_c):
      print("The model is hallucinating, skipping bar")
      return None

    upside = pred_h - current_price
    downside = current_price - pred_l
    if downside == 0: return None
    rr_ratio = upside/downside

    risk_per_trade = 0.01
    capital = 100000

    entry_price = current_price
    stop_loss = entry_price * 0.99  # 0.1%

    risk_amount = capital * risk_per_trade
    risk_per_share = entry_price - stop_loss

    self.position.size = int(risk_amount / risk_per_share)

    # Strategy: Trade only if there is significant profit ( >2% )
    # and if there is return to reward ratio of 3:1
    if (pred_c > current_price and rr_ratio >= 3.0):
      stop_price = pred_l * 0.99 # 1% lower than the predicted lowest
      target_price = pred_h * 0.99 # target price is 99% of pred_h
      self.buy(sl = stop_price, tp=target_price)
    else:
      self.position.close()
      self.sell()

bt = Backtest(data, Trading, comission=.02)

